# DOE Analysis for Beta Version

### National Renewable Energy Laboratory

#### Rob Hammond

##### 16 July 2020

In [1]:
import os
from time import perf_counter
from pprint import pprint

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.simulation import SwomEnvironment
from wombat.windfarm import Windfarm
from wombat.windfarm.system import System
from wombat.simulation.repairs import RepairManager
from wombat.simulation.transport import Equipment
from wombat.utilities import load_yaml

pd.set_option("display.max_rows", 1000)
%matplotlib inline

In [2]:
scenarios = {
    #### OFFSHORE ####
    "offshore_base_3_month_summer": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_3_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_summer.yaml"],
    "offshore_base_no_weather_3_month_summer": 
        ["vineyard_wind_weather_1998_2017_no_wind_wave.csv", "offshore_dudgeon_array.csv", "offshore_jackup_3_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_summer.yaml"],
    "offshore_double_3_month_summer": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array_double.csv", "offshore_jackup_3_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_summer.yaml"],
    "offshore_half_3_month_summer": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array_half.csv", "offshore_jackup_3_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_summer.yaml"],
    "offshore_12_month": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_12_month.yaml", "offshore_ctv.yaml", "cable_vessel_12_month.yaml"],
    "offshore_0_month": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", None, None, None],
    "offshore_base_2_month_summer": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_2_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_2_month_summer.yaml"],
    "offshore_base_no_weather_2_month_summer": 
        ["vineyard_wind_weather_1998_2017_no_wind_wave.csv", "offshore_dudgeon_array.csv", "offshore_jackup_2_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_2_month_summer.yaml"],
    "offshore_base_1_month_summer": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_1_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_1_month_summer.yaml"],
    "offshore_base_no_weather_1_month_summer": 
        ["vineyard_wind_weather_1998_2017_no_wind_wave.csv", "offshore_dudgeon_array.csv", "offshore_jackup_1_month_summer.yaml", "offshore_ctv.yaml", "cable_vessel_1_month_summer.yaml"],
    "offshore_base_3_month_fall": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_3_month_fall.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_fall.yaml"],
    "offshore_base_3_month_winter": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_3_month_winter.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_winter.yaml"],
    "offshore_base_3_month_spring": 
        ["vineyard_wind_weather_1998_2017.csv", "offshore_dudgeon_array.csv", "offshore_jackup_3_month_spring.yaml", "offshore_ctv.yaml", "cable_vessel_3_month_spring.yaml"],
    
    #### ONSHORE ####
    "onshore_base_3_month_summer": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_3_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_summer.yaml"],
    "onshore_base_no_weather_3_month_summer": 
        ["sweetwater_weather_1998_2017_no_wind_wave.csv", "onshore_dudgeon_array.csv", "onshore_crawler_3_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_summer.yaml"],
    "onshore_double_3_month_summer": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array_double.csv", "onshore_crawler_3_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_summer.yaml"],
    "onshore_half_3_month_summer": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array_half.csv", "onshore_crawler_3_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_summer.yaml"],
    "onshore_12_month": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_12_month.yaml", "onshore_onsite.yaml", "onshore_cable_12_month.yaml"],
    "onshore_0_month": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", None, None, None],
    "onshore_base_2_month_summer": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_2_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_2_month_summer.yaml"],
    "onshore_base_no_weather_2_month_summer": 
        ["sweetwater_weather_1998_2017_no_wind_wave.csv", "onshore_dudgeon_array.csv", "onshore_crawler_2_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_2_month_summer.yaml"],
    "onshore_base_1_month_summer": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_1_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_1_month_summer.yaml"],
    "onshore_base_no_weather_1_month_summer": 
        ["sweetwater_weather_1998_2017_no_wind_wave.csv", "onshore_dudgeon_array.csv", "onshore_crawler_1_month_summer.yaml", "onshore_onsite.yaml", "onshore_cable_1_month_summer.yaml"],
    "onshore_base_3_month_fall": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_3_month_fall.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_fall.yaml"],
    "onshore_base_3_month_winter": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_3_month_winter.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_winter.yaml"],
    "onshore_base_3_month_spring": 
        ["sweetwater_weather_1998_2017.csv", "onshore_dudgeon_array.csv", "onshore_crawler_3_month_spring.yaml", "onshore_onsite.yaml", "onshore_cable_3_month_spring.yaml"],
}

In [3]:
np.random.seed(0)
library_path = "/Users/rhammond/Documents/GitHub/wombat/library/"

print(f"{'Simulation Name':>42} | {'Run Time':>9} | {'Data Time':>10} | {'Total Time':>11} | {'N Events':>10} | {'N Operations':>12}")

def run_scenario(simulation_name, weather, layout, crane, crew_transfer, cable):
    
    print(f"{simulation_name:>42}", end=" | ")
    start = perf_counter()
    
    #### THE SETUP ####
    env = SwomEnvironment(library_path, weather, simulation_name=simulation_name, workday_start=8, workday_end=18)
    manager = RepairManager(env)
    windfarm = Windfarm(env, layout, manager)

    if crane is not None:
        crn = Equipment(env, windfarm, manager, crane)
    if crew_transfer is not None:
        ctv = Equipment(env,windfarm, manager, crew_transfer)
    if cable is not None:
        cab = Equipment(env, windfarm, manager, cable)
    
    #### RUN THE SCENARIO ####
    env.run()
    end1 = perf_counter()
    print(f"{(end1 - start) / 60:>9,.2f}", end=" | ")
    
    #### GATHER AND SAVSE THE RESULTS ####
    events = env.create_events_log_dataframe()
    operations = env.create_operations_log_dataframe()
    
    events_save_path = os.path.join(library_path, "outputs", "csv_logs", f"{simulation_name}_events.csv")
    events.sort_values("env_time").to_csv(events_save_path, index_label="datetime")
    
    turbine_columns = [col for col in operations if col not in ("env_datetime", "env_time", "DOW_OSS")]
    column_order = ["env_datetime", "env_time", "windfarm", "DOW_OSS"] + turbine_columns
    
    operations["windfarm"] = operations["DOW_OSS"].values * np.mean(operations[turbine_columns].values, axis=1)
    operations = operations[column_order]
    
    operations_save_path = os.path.join(library_path, "outputs", "csv_logs", f"{simulation_name}_operations.csv")
    operations.sort_values("env_time").to_csv(operations_save_path, index_label="datetime")
    
    end2 = perf_counter()
    print(f"{(end2 - end1) / 60:>10,.2f} | {(end2 - start) / 60:>11,.2f} | {events.shape[0]:>10,.0f} | {operations.shape[0]:>12,.0f}")
    
#     print(f"{simulation_name:>40} | {(end1 - start) / 60:>9,.2f} | {(end2 - end1) / 60:>10,.2f} | {(end2 - start) / 60:>11,.2f}")
    
for simulation_name, values in scenarios.items():

    events_save_path = os.path.join(library_path, "outputs", "csv_logs", f"{simulation_name}_events.csv")
    operations_save_path = os.path.join(library_path, "outputs", "csv_logs", f"{simulation_name}_operations.csv")
    if os.path.isfile(events_save_path) and os.path.isfile(operations_save_path):
        continue

    weather, layout, crane, crew_transfer, cable = values
    run_scenario(simulation_name, weather, layout, crane, crew_transfer, cable)

                           Simulation Name |  Run Time |  Data Time |  Total Time |   N Events | N Operations
                          onshore_12_month |      3.86 |       0.60 |        4.46 |     45,757 |      175,320
                           onshore_0_month |      5.09 |       0.52 |        5.61 |      3,233 |      175,320
